In [ ]:
import pandas as pd
import numpy as np
import json
from pathlib import Path
from itables import init_notebook_mode, show
import matplotlib.pyplot as plt
import seaborn as sns
#pd.set_option('display.float_format',  '{:18,.0}'.format)
pd.set_option('display.float_format', lambda x: f'{x:15,.0f}' if abs(x)>1e5 else f'{x:15.2f}')
import itables

# Initialize interactive display mode
itables.init_notebook_mode(all_interactive=True)


weight = pd.read_parquet(f'../data/weight_base_import_2022.parquet')
# Load Excel file
# Load Excel file with HS codes as strings
hs_conversion = pd.read_excel(
    '../cat/HS2022toHS2017ConversionAndCorrelationTables.xlsx',
    sheet_name='HS2022-HS2017 Correlations',
    dtype=str  # Ensures all columns are read as strings
)

# Rename columns
hs_conversion.rename(columns={'From HS2022': 'hs6_2022', 'To HS2017': 'hs6_2017', 'Relationship': 'Relationship'}, inplace=True)

# Sort by commodity
hs_conversion.sort_values(by='hs6_2017', inplace=True)

# equal 2022 and 2017
hs_conversion['equal'] = hs_conversion['hs6_2022'] == hs_conversion['hs6_2017']


# Save as Parquet
hs_conversion.to_parquet(f'../cat/hs_correspondance_2017_2022.parquet', index=False)


# Load base price data
base_price = pd.read_parquet(f'../data/base_priceimport_2022.parquet')
# Ensure comno is a string before slicing
base_price['comno6'] = base_price['comno'].astype(str).str[:6]


base_price

In [ ]:
print(hs_conversion.columns)
print(base_price.columns)
weight

In [ ]:
# Merge the datasets on the correct columns
merged_data = hs_conversion.merge(base_price, left_on='hs6_2022', right_on='comno6', how='left', indicator='found_hs')

# Map the 'found_hs' indicator values to 1 and 0
merged_data['found_hs'] = merged_data['found_hs'].map({'both': 1, 'left_only': 0})

merged_data = merged_data.merge(weight, left_on='comno', right_on='comno', how='left')

#merged_data_filtered = merged_data[merged_data['Relationship'] == '1:1']


# Display the merged data
#merged_data_filtered

crosstab = pd.crosstab(merged_data['found_hs'], columns='Frequency', margins=True)
crosstab

In [ ]:
merged_data_filtered2 = merged_data[merged_data['found_hs'] == 1]
merged_data_filtered3 = merged_data_filtered2[merged_data_filtered2['equal'] == 0]
merged_data_filtered3.loc[merged_data_filtered3['hs6_2017'] == '040310']